In [3]:

import torch
from torch_snippets import *
#from torch import nn
import numpy as np
from torch import nn
from PIL import Image
from fastapi import Request


class SDD(nn.Module):
    classes = ['defect','non_defect']
    def __init__(self, model, device = 'cpu'):
        super().__init__()
        self.model = model.to(device)
        self.device = device

    @torch.no_grad()
    def forward(self, x):
        x = x.view(-1,3, 224,224).to(device)
        pred = self.model(x)
        conf = pred[0][0]
        clss = np.where(conf.item()<0.5, 'non_defect', 'defect')
        print(clss)
        return clss.item()
    
    def predict(self, input):
        print(input)
        if isinstance(input,( PIL.PngImagePlugin.PngImageFiles, PIL.Image.Image)):
            im = (255*np.array(input)).astype(np.uint8)
        else:
            im = read(input, 1)
        print(im.shape)

        im = resize(img,224)
        im = torch.tensor(im/255)
        im = im.permute(2,0,1).float()
        clss = self.forward(im)
        return {"class": clss}
    predict_from_path =  predict
    predict_from_image = predict

In [5]:
import os
print(os.getcwd())

C:\Users\matyp\Documents\GitHub\object-detection


In [4]:
import os , io, torch
#from ssd import SSD
from torch_snippets import P, makedir
from PIL import Image
from fastapi import FastAPI, Request, UploadFile, File
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates


#cargar el modelo desde sdd.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SDD(torch.load('sdd.weights.pth', map_location=device))

server_root = P('/tmp')
templates = './templates'
static = server_root/'server/statics'
files = server_root/'server/files'
for fldr in [static,files]: makedir(fldr)

app = FastAPI()
app.mount("/static", StaticFiles(directory=static), name="static")
app.mount("/files", StaticFiles(directory=files), name="files")
templates = Jinja2Templates(directory=templates)

@app.get("/")
async def read_item(request: Request):
    return templates.TemplateResponse("home.html",{"request": request})
@app.post('/uploaddata/')
async def upload_file(request:Request, file:UploadFile=File(...)):
    print(request)
    content = file.read()
    saved_filepath= f'{files}/{file.filename}'
    with open(saved_filepath,'wb') as f:
        f.write(content)
    output = model.predict_from_path(saved_filepath)
    payload = {
        'request': request,
        "filename": file.filename,
        'output': output
    }
    return templates.TemplateResponse("home.html", payload)





@app.post("/predict")
def predict(request: Request, file: UploadFile=File(...)):
    content = file.file.read()
    image = Image.open(io.BytesIO(content))
    output = model.predict_from_image(image)
    return output

AssertionError: Routed paths must start with '/'

In [7]:
import os
print(os.getcwd())

C:\Users\matyp\Documents\GitHub\object-detection
